<a href="https://colab.research.google.com/github/saiteja-ms/Assignment/blob/main/Task_1_and_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wikipedia-api beautifulsoup4 requests argparse


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14347 sha256=da3612139bac1ecd8d9622acf2016d122422e396ec61a8499f534ba20a37eac8
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import wikipediaapi

In [2]:
def get_search_query():
    # If running in a notebook or Colab, use a default query
    if 'google.colab' in sys.modules or 'ipykernel' in sys.modules:
        return "Artificial Intelligence"  # You can change this default search term
    else:
        import argparse
        parser = argparse.ArgumentParser(description="Find the closest Wikipedia article to a given topic.")
        parser.add_argument('query', type=str, help="The search query (topic) to find the closest Wikipedia article.")
        args = parser.parse_args()
        return args.query

In [3]:
def get_wikipedia_article(query):
    wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        user_agent='My_Wikipedia_Bot/1.0 (My_Email@example.com)'
    )
    page = wiki_wiki.page(query)
    if page.exists():
        return page.text
    else:
        return None

In [4]:
def save_article_to_file(content, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)

In [5]:
def main():
    # Get the search query from the command line or default value
    query = get_search_query()

    # Get the Wikipedia article closest to the query
    article_content = get_wikipedia_article(query)

    if article_content:
        # Save the content to a file
        save_article_to_file(article_content, f'{query}_wikipedia_article.txt')
        print(f'Article saved successfully as {query}_wikipedia_article.txt')
    else:
        print(f"No Wikipedia article found for '{query}'.")


In [6]:
if __name__ == "__main__":
    main()


Article saved successfully as Artificial Intelligence_wikipedia_article.txt


# Task - 2


In [7]:
!pip install faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 15.8 MB/s eta 0:00:00


In [8]:
import faiss
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [9]:
#Load the pre-trained SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Function to load the article from the text file
def load_article(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return f.read()

In [11]:
# Function to chunk the text
def chunk_text(text, chunk_size=500, overlap=False):
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size):
        chunk = words[i:i + chunk_size] if not overlap else words[i:i + chunk_size] + words[i - (chunk_size // 2): i]
        chunks.append(" ".join(chunk))

    return chunks

In [12]:
# Function to create embeddings for the chunks
def create_embeddings(chunks):
    return model.encode(chunks)

In [13]:
# Function to store embeddings in a FAISS vector database
def store_embeddings_in_faiss(embeddings, filename):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)

    # Save the FAISS index to a file
    faiss.write_index(index, filename)

In [14]:
def main():
    # Load the article content
    article_filename = 'Artificial Intelligence_wikipedia_article.txt'  # Replace with your article's filename
    article_content = load_article(article_filename)

    # Chunk the article
    chunk_size = 500  # You can adjust this based on your requirements
    chunks = chunk_text(article_content, chunk_size=chunk_size, overlap=True)  # Using overlap for better context retention

    # Generate embeddings for the chunks
    embeddings = create_embeddings(chunks)

    # Store embeddings in FAISS vector database
    faiss_filename = 'wikipedia_article_faiss.index'
    store_embeddings_in_faiss(embeddings, faiss_filename)

    print(f"FAISS vector database saved successfully as {faiss_filename}")

if __name__ == "__main__":
    main()

FAISS vector database saved successfully as wikipedia_article_faiss.index
